
6- El GCPD (Gotham City Police Dept) recolecta la información de casos policiales que acontecen en Ciudad Gótica. Esta información se encuentra guardada en un dataframe con el siguiente formato: (fecha, id_caso, descripcion, estado_caso, categoria, latitud, longitud). Los posibles estados que puede tener un caso son 1: caso abierto, 2: caso resuelto, 3: cerrado sin resolución. Las fechas se encuentran en el formato YYYY-MM-DD. Por otro lado el comisionado Gordon guarda un registro detallado sobre en cuáles casos fue activada la batiseñal para pedir ayuda del vigilante, Batman. Esta información se encuentra en un Dataframe con el siguiente formato (id_caso, respuesta), siendo campo respuesta si la señal tuvo una respuesta positiva (1) o negativa (0) de parte de él El sector encargado de las estadísticas oficiales del GCPD quiere con esta información analizar las siguientes situaciones: 

* Tasa de resolución de casos de la fuerza policial por categoría de caso (considerando aquellos casos en los que no participó Batman).
* Tasa de resolución de casos con la ayuda de Batman (considerando que aquellos casos en los que fue llamado con la batiseñal, participó en la resolución). 
* Indicar el mes del año pasado en el que Batman tuvo mayor participación en la investigación de casos. 


In [ ]:
import pandas as pd
import numpy as np

#------------Constantes--------#
BATMAN_NO_PARTICIPO = 0
BATMAN_PARTICIPO = 1
CASO_RESUELTO = 3

In [ ]:
df_casos = pd.read_csv("/DataSets/casos.csv")
df_casos.dropna(inplace=True)
display(df_casos)

df_respuestas = pd.read_csv("/DataSets/respuestaBatman.csv")
df_respuestas.dropna(inplace=True)
display(df_respuestas)


,fecha,id_caso,descripcion,estado_caso,categoria,latitud,longitud
0,2020/05/03/,100,"vestibulum lorem, sit",2,asesinato,68,16
2,2020/02/02/,102,aliquet magna a neque. Nullam,2,secuestro,29,27
3,2020/05/13/,103,Sed,2,secuestro,58,48
5,2019/10/29/,105,ipsum. Donec sollicitudin adipiscing,3,secuestro,65,90
7,2020/04/08/,107,lobortis quam,2,secuestro,60,32
...,...,...,...,...,...,...,...
93,2019/11/10/,193,per inceptos hymenaeos. Mauris ut,2,asesinato,77,19
94,2020/02/04/,194,Suspendisse non leo. Vivamus,2,asesinato,51,74
96,2020/07/02/,196,"leo, in",3,secuestro,75,30
98,2020/03/20/,198,velit justo nec ante.,2,robo,85,87


,id_caso,respuesta
0,100,0
1,101,0
2,102,0
3,103,0
4,104,1
...,...,...
95,195,1
96,196,0
97,197,1
98,198,0


* Tasa de resolución de casos de la fuerza policial por categoría de caso (considerando aquellos casos en los que no participó Batman).

Primero debemos quedarnos con aquellos casos en ls que Bataman no participo. Para esto veremos en el df_respuestas aquellas filas que tengan como respuesta 0 de batman.
Luego de haber hecho esto, uniremos ambos df y nos quedaremos solo con los que cumplan la condicion de que batman no haya respondido el caso.

In [ ]:
df_batman_no_participa = df_respuestas[df_respuestas['respuesta'] == BATMAN_NO_PARTICIPO]
df_merge = df_casos.merge(df_batman_no_participa,how='left',on='id_caso').dropna()
df_merge.head()

,fecha,id_caso,descripcion,estado_caso,categoria,latitud,longitud,respuesta
0,2020/05/03/,100,"vestibulum lorem, sit",2,asesinato,68,16,0.0
1,2020/02/02/,102,aliquet magna a neque. Nullam,2,secuestro,29,27,0.0
2,2020/05/13/,103,Sed,2,secuestro,58,48,0.0
4,2020/04/08/,107,lobortis quam,2,secuestro,60,32,0.0
6,2020/05/06/,110,mauris ut mi.,2,robo,77,98,0.0


Siguiendo debemos separar por las diferentes categorias de casos que tiene el df.

In [ ]:
df_merge.groupby('categoria')['estado_caso'].agg(lambda x: (x == CASO_RESUELTO).mean()).rename('tasa de resolución')

categoria
asesinato    0.111111
robo         0.500000
secuestro    0.235294
Name: tasa de resolución, dtype: float64

* Tasa de resolución de casos con la ayuda de Batman (considerando que aquellos casos en los que fue llamado con la batiseñal, participó en la resolución). 

Similar a lo que hicimos anteriormente, salvo que no debemos agrupar por las distintas categorias.

In [ ]:
df_participa_batman = df_respuestas[df_respuestas['respuesta'] == BATMAN_PARTICIPO]
df_merge2 = df_casos.merge(df_participa_batman,how='left',on='id_caso').dropna()
df_merge2.head()

,fecha,id_caso,descripcion,estado_caso,categoria,latitud,longitud,respuesta
3,2019/10/29/,105,ipsum. Donec sollicitudin adipiscing,3,secuestro,65,90,1.0
5,2020/06/11/,108,ante bibendum ullamcorper. Duis,1,secuestro,92,84,1.0
7,2020/07/10/,111,ipsum leo elementum,1,secuestro,29,77,1.0
11,2019/11/19/,118,dignissim tempor,3,asesinato,49,79,1.0
14,2020/05/28/,121,"vitae,",2,secuestro,83,99,1.0


In [ ]:
cantidad_casos_totales = len(df_merge2['estado_caso'])
casos_resueltos = len(df_merge2[df_merge2['estado_caso'] == CASO_RESUELTO])
print("La tasa de casos resueltos es de : ",round(casos_resueltos / cantidad_casos_totales,2),"%")

La tasa de casos resueltos es de :  0.42 %


Otra forma de resolverlo

In [ ]:
df_merge2['estado_caso'].apply(lambda x: x==CASO_RESUELTO).mean()

0.4166666666666667

* Indicar el mes del año pasado en el que Batman tuvo mayor participación en la investigación de casos. 